<a href="https://colab.research.google.com/github/younes-sadi/Task-image-segmentation/blob/main/Dataset_image_segmentation_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import os

# Upload kaggle.json
uploaded = files.upload()

# Set up Kaggle API
os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# Install Kaggle API
!pip install -q kaggle

# Verify authentication
!kaggle datasets list

# Download dataset
!kaggle datasets download -d mahmoudnoor/high-resolution-catdogbird-image-dataset-13000

# Unzip the dataset
!unzip -q high-resolution-catdogbird-image-dataset-13000.zip -d cat_dog_bird_dataset

Saving kaggle.json to kaggle.json
ref                                                         title                                          size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  --------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
atharvasoundankar/chocolate-sales                           Chocolate Sales Data 📊🍫                        14KB  2025-03-19 03:51:40          20072        329  1.0              
adilshamim8/student-depression-dataset                      Student Depression Dataset                    456KB  2025-03-13 03:12:30          10873        158  1.0              
khushikyad001/air-pollution-in-china-2015-2025              Air Pollution in China (2015-2025)            443KB  2025-03-31 06:21:35           1256         23  1.0              
amansingh0000000/flipkart-product-list                      Flipkart Product

In [ ]:
import os
import shutil

# Define paths
original_dataset_dir = '/content/cat_dog_bird_dataset'
subset_dir = '/content/subset'

# Classes to process
classes = ['cat', 'dog', 'bird']

# Create subset directories
for cls in classes:
    os.makedirs(os.path.join(subset_dir, cls), exist_ok=True)

# Number of images to copy per class
n_images = 500

for cls in classes:
    # Path to original class subdirectory
    src_dir = os.path.join(original_dataset_dir, cls, cls)

    # List all files in the nested directory
    files = os.listdir(src_dir)

    # Select first 800 images
    selected_files = files[:n_images]

    # Copy files to subset directory
    for file in selected_files:
        src = os.path.join(src_dir, file)
        dst = os.path.join(subset_dir, cls, file)
        shutil.copy(src, dst)

# Verify counts
for cls in classes:
    subset_cls_dir = os.path.join(subset_dir, cls)
    print(f"Class {cls}: {len(os.listdir(subset_cls_dir))} images")

Class cat: 500 images
Class dog: 500 images
Class bird: 500 images


In [ ]:
!pip install torch torchvision opencv-python matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import os
import cv2
import torch
import torchvision
import numpy as np
from torchvision.models.detection import maskrcnn_resnet50_fpn
from tqdm import tqdm
# Initialize Mask R-CNN model (pretrained on COCO)
model = maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

# COCO class labels (only relevant classes listed)
COCO_CLASSES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

# Paths
dataset_root = "/content/subset"  #  subset folder with cat/dog/bird
mask_root = "/content/subset/masks"                   # Output directory for masks

# Create mask directories
classes = ['cat', 'dog', 'bird']
for cls in classes:
    os.makedirs(os.path.join(mask_root, cls), exist_ok=True)

def generate_masks(class_name):
    # Paths for input images and output masks
    image_dir = os.path.join(dataset_root, class_name)
    mask_dir = os.path.join(mask_root, class_name)

    # List all image files
    image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Process each image
    for img_file in tqdm(image_files, desc=f"Processing {class_name}"):
        image_path = os.path.join(image_dir, img_file)
        mask_path = os.path.join(mask_dir, img_file)  # Same name as input

        # Skip if mask already exists
        if os.path.exists(mask_path):
            continue

        # Load image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Could not read {image_path}")
            continue

        # Convert to RGB
        try:
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except:
            print(f"Error converting {image_path}")
            continue

        # Generate mask
        image_tensor = torchvision.transforms.functional.to_tensor(image_rgb)
        with torch.no_grad():
            predictions = model([image_tensor])

        # Filter predictions for the target class (e.g., 'cat' when processing cat folder)
        target_class = class_name  # e.g., 'cat', 'dog', 'bird'
        masks = []

        for i in range(len(predictions[0]['labels'])):
            label = COCO_CLASSES[predictions[0]['labels'][i].item()]
            if label in ['cat', 'dog', 'bird']:
               mask = predictions[0]['masks'][i][0].numpy()
               masks.append(mask)

        # Combine masks (if multiple instances)
        if len(masks) > 0:
            combined_mask = np.clip(np.sum(masks, axis=0), 0, 1)
        else:
            combined_mask = np.zeros(image.shape[:2], dtype=np.float32)  # Blank mask

        # Save mask as PNG
        mask_uint8 = (combined_mask * 255).astype(np.uint8)
        cv2.imwrite(mask_path, mask_uint8)

# Generate masks for all classes
for cls in classes:
    generate_masks(cls)

print("All masks generated! ")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:01<00:00, 96.7MB/s]
Processing bird: 100%|██████████| 500/500 [1:07:10<00:00,  8.06s/it]

All masks generated! 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
filename = "Dataset_image_segmentation_task.ipynb"
import IPython
IPython.display.display(IPython.display.Javascript('IPython.notebook.save_checkpoint();'))


<IPython.core.display.Javascript object>

In [ ]:
!ls


drive  sample_data  Task-image-segmentation


In [ ]:
!git config --global user.email "sadi.belkacem@outlook.com"
!git config --global user.name "younes-sadi"


In [ ]:
!git clone https://github.com/younes-sadi/Task-image-segmentation.git

Cloning into 'Task-image-segmentation'...


In [ ]:
!mv /content/drive/MyDrive/ColabNotebooks/Dataset_image_segmentation_task.ipynb Task-image-segmentation/


In [ ]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [ ]:
!cd /content/Task-image-segmentation
!git add .
!git commit -m "Add dataset notebook"
!git push -u origin main

error: 'Task-image-segmentation/' does not have a commit checked out
fatal: adding files failed
On branch main

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	Task-image-segmentation/
	drive/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/younes-sadi/Task-image-segmentation.git'


In [ ]:
!git remote add origin https://github.com/younes-sadi/Task-image-segmentation.git

In [ ]:
!git branch -M main

In [ ]:
!cd /content/Task-image-segmentation
!git init
!git add /content/Task-image-segmentation/Dataset_image_segmentation_task.ipynb
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/younes-sadi/Task-image-segmentation.git
!git push -u origin main

Reinitialized existing Git repository in /content/.git/
On branch main

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	Task-image-segmentation/
	drive/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)
error: remote origin already exists.
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/younes-sadi/Task-image-segmentation.git'
